In [ ]:
import numpy as np

# Module 

**Module** is like a black box, which operates forward and backward propagation.

In [ ]:
class Module(object):
    """
    You can refer to Module as an abstract class. You need to override all the methods in inherited classes.
    """

    def __init__ (self):
        """
        output = module.forward(input)    
        gradInput = module.backward(input, gradOutput)
        """
        self.output = None
        self.gradInput = None
        self.train = True

    def forward(self, inpt):
        """
        Performs forward-propagation: takes an input, and computes the corresponding output 
        (see updateOutput method).
        """
        return self.updateOutput(inpt)

    def backward(self, inpt, gradOutput):
        """
        Performs backpropagation step through the module, with respect to the given input.
        
        This includes 
         - computing a local gradient w.r.t. input (see updateGradInput),
         - computing a gradient w.r.t. parameters to update parameters while optimizing (see accGradParameters)
        """
        self.updateGradInput(inpt, gradOutput)
        self.accGradParameters(inpt, gradOutput)
        return self.gradInput
    

    def updateOutput(self, inpt):
        """
        Stores and returns the output using the current parameter set of the class and input. 
        Used in the forward propagation.
        """
        # just some easy case        
        self.output = inpt 
        return self.output
        
        pass

    def updateGradInput(self, inpt, gradOutput):
        """
        Computing the gradient of the module with respect to its own input. 
        This is returned in `gradInput`. Also, the `gradInput` state variable is updated accordingly.
        
        The shape of `gradInput` is always the same as the shape of `input`.
        
        Make sure to both store the gradients in `gradInput` field and return it.
        """
        
        # just some easy case
        self.gradInput = gradOutput 
        return self.gradInput
        
        pass   
    
    def accGradParameters(self, inpt, gradOutput):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass
    
    def zeroGradParameters(self): 
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass
        
    def getParameters(self):
        """
        Returns a list with its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
        
    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
    
    def training(self):
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = True
    
    def evaluate(self):
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = False
    
    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return "Module"

# Sequential container

Define forward and backward propagation workflows here.

In [ ]:
class Sequential(Module):
    """
    Refer to Sequential as a container, containing all the modules (layers) sequentially.
    Input is processed by each module (layer) in self.modules consecutively.
    The resulting array is the Output. 
    """
    
    def __init__ (self):
        super(Sequential, self).__init__()
        self.modules = []
   
    def add(self, module):
        """
        Adds a module (layer) to the container.
        """
        self.modules.append(module)

    def updateOutput(self, inpt):
        """
        Main workflow of forward-propagation (this method is called in the forward method of base class):
        
            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})   
            
        """
        self.y=[]   
        
        #TODO: implement forward pass described in docstring
        
        return self.output

    def backward(self, inpt, gradOutput):
        """
        Main workflow of backward-propagation:
            
            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)   
            gradInput = module[0].backward(input, g_1)   
             
        Each module takes the input seen during the forward pass (y_{i-1}) 
        and computes corresonding gradient (g_{i}).   
        """
        y = self.y
        g = []
        
        #TODO: implement backward pass described in docstring
        
        return self.gradInput
      

    def zeroGradParameters(self): 
        for module in self.modules:
            module.zeroGradParameters()
    
    def getParameters(self):
        """
        Should gather all parameters in a list.
        """
        return [x.getParameters() for x in self.modules]
    
    def getGradParameters(self):
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        return [x.getGradParameters() for x in self.modules]
    
    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules])
        return string
    
    def __getitem__(self,x):
        return self.modules.__getitem__(x)

# Layers

In [ ]:
class Linear(Module):
    """
    Performs pre-activation thus outputting weighted input.
    
    The module should work with batch input 2D input of shape (n_samples, n_features).
    """
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()
        
        #TODO: Initialize biases and weights from standard normal distribution
        
        #self.b = 
        #self.W = 
    
        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)
        
    def updateOutput(self, inpt):
        
        #TODO: calculate output of linear layer
        
        return self.output

    def updateGradInput(self, inpt, gradOutput):
        
        self.gradInput = gradOutput.dot(self.W)
        return self.gradInput
    
    def accGradParameters(self, inpt, gradOutput):
        
        self.gradW = gradOutput.T.dot(inpt)
        self.gradb = gradOutput.sum(axis=0)
    
    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)
        
    def getParameters(self):
        return [self.W, self.b]
    
    def getGradParameters(self):
        return [self.gradW, self.gradb]
    
    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q

In [ ]:
class SoftMax(Module):
    def __init__(self):
         super(SoftMax, self).__init__()
    
    def updateOutput(self, inpt):
        self.output = (np.exp(inpt) / np.sum(np.exp(inpt), axis =1, keepdims = True))
        
        return self.output
    
    def updateGradInput(self, inpt, gradOutput):
        self.gradInput = self.output*(gradOutput - (np.sum(gradOutput*self.output, axis=1, keepdims = True)))
        
        return self.gradInput
    
    def __repr__(self):
        return "SoftMax"

# Activations

In [ ]:
class Sigmoid(Module):
    def __init__(self):
         super(Sigmoid, self).__init__()
    
    def updateOutput(self, inpt):
        
        #TODO: implement sigmoid function

        return self.output
    
    def updateGradInput(self, inpt, gradOutput):
        
        #TODO: implement gradient of sigmoid function
        
        return self.gradInput
    
    def __repr__(self):
        return "Sigmoid"
    


In [ ]:
class Tanh(Module):
    def __init__(self):
         super(Tanh, self).__init__()
    
    def updateOutput(self, inpt):
        
        self.output = (1. - np.exp(-2*inpt)) / (1. + np.exp(-2*inpt))
        return self.output
    
    def updateGradInput(self, inpt, gradOutput):
        
        self.gradInput = gradOutput*(1-self.output**2)
        return self.gradInput
    
    def __repr__(self):
        return "Tanh"

In [ ]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()
    
    def updateOutput(self, inpt):
        
        #TODO: implement RELU function

        return self.output
    
    def updateGradInput(self, inpt, gradOutput):
        
        self.gradInput = gradOutput*(inpt>0)
        return self.gradInput
    
    def __repr__(self):
        return "ReLU"

# Criterions

In [ ]:
class Criterion(object):
    """
    Base class for criterions.
    """
    def __init__ (self):
        self.output = None
        self.gradInput = None
    
    def forward(self, inpt, target):
        """
        Given an input and a target, compute the loss function 
        associated to the criterion and return the result.
            
        (all the code goes in updateOutput)
        """
        return self.updateOutput(inpt, target)

    def backward(self, inpt, target):
        """
        Given an input and a target, compute the gradients of the loss function
        associated to the criterion and return the result. 

        (all the code goes in updateGradInput)
        """
        return self.updateGradInput(inpt, target)
    
    def updateOutput(self, inpt, target):
        """
        Override this.
        """
        return self.output

    def updateGradInput(self, inpt, target):
        """
        Override this.
        """
        return self.gradInput   

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return "Criterion"

**MSECriterion** (L2 norm)

In [ ]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()

    def updateOutput(self, inpt, target):   
        
        #TODO: implement MSE
        
        return self.output 
 
    def updateGradInput(self, inpt, target):
        
        #TODO: calculate gradient w.r.t input

        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

**CrossEntropyCriterion**. ([multiclass log loss](http://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html#cross-entropy)). Remember that targets are one-hot encoded.

In [ ]:
class CrossEntropyCriterion(Criterion):
    def __init__(self):
        super(CrossEntropyCriterion, self).__init__()
    
    def updateOutput(self, inpt, target): 
        
        # trick to avoid numerical errors
        input_clamp = np.maximum(1e-15, np.minimum(inpt, 1 - 1e-15) )

        self.output = np.sum(target*np.log(input_clamp))/(-len(input_clamp))
        
        return self.output

    def updateGradInput(self, inpt, target):
        
        input_clamp = np.maximum(1e-15, np.minimum(inpt, 1 - 1e-15))
                
        #TODO: calculate gradient w.r.t input
        
        return self.gradInput
    
    def __repr__(self):
        return "CrossEntropyCriterion"